<a href="https://colab.research.google.com/github/esaghapour/Data/blob/main/CHOP_Data_Exercise___Ehsan_Saghapour%2C_PH_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*The code is written by **Ehsan Saghapour, Ph.D**. This code shows the style of my programming language.*

#**Clone the GitHub repository into the Google Drive**

In [30]:
!git clone https://github.com/chop-analytics/analyst-take-home-task

fatal: destination path 'analyst-take-home-task' already exists and is not an empty directory.


# **Import packages**

In [31]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# **Read Datasets**

In [32]:
allergies= pd.read_csv('/content/analyst-take-home-task/datasets/allergies.csv')
encounters= pd.read_csv('/content/analyst-take-home-task/datasets/encounters.csv')
medications= pd.read_csv('/content/analyst-take-home-task/datasets/medications.csv')
patients= pd.read_csv('/content/analyst-take-home-task/datasets/patients.csv')
procedures= pd.read_csv('//content/analyst-take-home-task/datasets/procedures.csv')

In [33]:
patients=patients.set_index('Id')
medications=medications.set_index('PATIENT')

# **Part 1: Assemble the project cohort**

1.   The patient’s visit is an encounter for drug overdose.
2.   The hospital encounter occurs after July 15, 1999
3.   The patient’s age at time of encounter is between 18 and 35 (Patient is considered to be 35 until turning 36)

In [34]:
encounters=encounters[(encounters.DESCRIPTION == 'Drug rehabilitation and detoxification')]  # The patient’s visit is an encounter for drug overdose
encounters['START_STOP']=(pd.to_datetime(encounters['STOP']).dt.date-pd.to_datetime(encounters['START']).dt.date)/np.timedelta64(1, 'D')
encounters['START_STOP']=np.round(encounters['START_STOP'])
encounters = encounters.loc[(encounters['START'] > '1999-07-15')]  # after July 15, 1999
encounters=encounters.sort_values(by=['START'])
filtered_encounters=encounters.drop_duplicates(subset=['PATIENT'], keep='first')
filtered_encounters=filtered_encounters.set_index('PATIENT')
encounters=encounters.set_index('PATIENT')

combine_encounters_patients1= pd.merge(patients,filtered_encounters,how='inner',left_index=True,right_index=True)

combine_encounters_patients1['age']=(pd.to_datetime(combine_encounters_patients1['START']).dt.date-pd.to_datetime(combine_encounters_patients1['BIRTHDATE']).dt.date)/np.timedelta64(1, 'Y')

combine_encounters_patients1 = combine_encounters_patients1.loc[(combine_encounters_patients1['age'] >= 18) & (combine_encounters_patients1['age']<36)]  #The patient’s age at time of encounter is between 18 and 35 

combine_encounters_patients1=combine_encounters_patients1[['START','STOP','age','DEATHDATE','START_STOP','Id']]

combine_encounters_patients1['age2']=(pd.to_datetime(combine_encounters_patients1['DEATHDATE']).dt.date-pd.to_datetime(combine_encounters_patients1['STOP']).dt.date)/np.timedelta64(1, 'D')

In [35]:
combine_encounters_patients1['Last_Day_STOP']=combine_encounters_patients1['STOP']
out=[]
for name in combine_encounters_patients1.index:
    idx=encounters.index==name
    ss=encounters[['START','STOP']][idx]
    ss=ss.sort_values(by=['START'])
    idx1=combine_encounters_patients1.index==name
    combine_encounters_patients1['Last_Day_STOP'][idx1]=ss['STOP'][-1]


# **Part 2: Create additional fields**

1. **DEATH_AT_VISIT_IND:**  1 if patient died during the drug overdose encounter, 0 if the patient died at a different time


In [36]:
combine_encounters_patients1['DEATH_AT_VISIT_IND']=(pd.to_datetime(combine_encounters_patients1['DEATHDATE']).dt.date-pd.to_datetime(combine_encounters_patients1['Last_Day_STOP']).dt.date)/np.timedelta64(1, 'D')
idx=combine_encounters_patients1['DEATH_AT_VISIT_IND']==0
combine_encounters_patients1['DEATH_AT_VISIT_IND'][idx]=1
combine_encounters_patients1['DEATH_AT_VISIT_IND'][~idx]=0

2.**COUNT_CURRENT_MEDS:** Count of active medications at the start of the drug overdose encounter

In [37]:
combine_encounters_patients1['COUNT_CURRENT_MEDS']=0
out=[]
for name in combine_encounters_patients1.index:
    idx=medications.index==name
    ss=medications[['DESCRIPTION']][idx]
    
    combine_encounters_patients1['COUNT_CURRENT_MEDS'][name]=len(np.unique(ss))

3. **CURRENT_OPIOID_IND:** 1 if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List (provided below), 0 if not. 

**Opioids List**:
*   Hydromorphone 325Mg
*   Fentanyl – 100 MCG
*   Oxycodone-acetaminophen 100 Ml


In [38]:
combine_encounters_patients1['CURRENT_OPIOID_IND']=0
out=[]
for name in combine_encounters_patients1.index:
    idx=medications.index==name
    ss=medications[['DESCRIPTION']][idx]
    k=(ss == 'Hydromorphone 325 MG') | (ss== 'Fentanyl 100 MCG') | (ss=='Oxycodone-acetaminophen 100ML') 

    if len(np.where(k==True)[0])>0:
       combine_encounters_patients1['CURRENT_OPIOID_IND'][name]=1
    else:
      combine_encounters_patients1['CURRENT_OPIOID_IND'][name]=0

**----------------------------------------------------------------------------------------------------------------**


4. **READMISSION_90_DAY_IND:** 1 if the visit resulted in a subsequent drug overdose readmission within 90 days, 0 if not
5. **READMISSION_30_DAY_IND:** 1 if the visit resulted in a subsequent drug overdose readmission within 30 days, 0 if not overdose encounter, 0 if not
6. **FIRST_READMISSION_DATE:** The date of the index visit's first readmission for drug overdose. Field should be left as N/A if no readmission for drug overdose within 90 days

In [39]:
combine_encounters_patients1['FIRST_READMISSION_DATE']=combine_encounters_patients1['STOP']
combine_encounters_patients1['READMISSION_90_DAY_IND']=0
combine_encounters_patients1['READMISSION_30_DAY_IND']=0

In [40]:
out=[]
for name in combine_encounters_patients1.index:
    idx=encounters.index==name
    ss=encounters[['START','STOP']][idx]
    ss=ss.sort_values(by=['START'])
    ss['START1']=ss['START']
    ss['START1'][0:-1]=ss['START'][1:]
    ss['final']=(pd.to_datetime(ss['START1']).dt.date-pd.to_datetime(ss['STOP']).dt.date)/np.timedelta64(1, 'D')
    idx1=combine_encounters_patients1.index==name
    combine_encounters_patients1['FIRST_READMISSION_DATE'][idx1]=ss['START1'][0]
    combine_encounters_patients1['START'][idx1]=ss['START'][0]
    if len(np.where(ss['final'] <=90)[0])>0 :
        combine_encounters_patients1['READMISSION_90_DAY_IND'][idx1]=1
    if (ss['final'][0] >90 or len(ss['final'])==1) :      
        combine_encounters_patients1['FIRST_READMISSION_DATE'][idx1]=np.nan
    if len(np.where(ss['final'] <=30)[0])>0 :
        combine_encounters_patients1['READMISSION_30_DAY_IND'][idx1]=1


In [41]:
combine_encounters_patients1 = combine_encounters_patients1.rename(columns={'START': 'HOSPITAL_ENCOUNTER_DATE', 'age': 'AGE_AT_VISIT', 'Id': 'ENCOUNTER_ID'})
combine_encounters_patients1.index.names = ['PATIENT_ID']

In [46]:
combine_encounters_patients1=combine_encounters_patients1[['ENCOUNTER_ID', 'HOSPITAL_ENCOUNTER_DATE', 'AGE_AT_VISIT',
       'DEATH_AT_VISIT_IND', 'COUNT_CURRENT_MEDS', 'CURRENT_OPIOID_IND', 'READMISSION_90_DAY_IND','FIRST_READMISSION_DATE']]
combine_encounters_patients1['DEATH_AT_VISIT_IND']=combine_encounters_patients1['DEATH_AT_VISIT_IND'].astype('int')

In [48]:
combine_encounters_patients1.head()

,ENCOUNTER_ID,HOSPITAL_ENCOUNTER_DATE,AGE_AT_VISIT,DEATH_AT_VISIT_IND,COUNT_CURRENT_MEDS,CURRENT_OPIOID_IND,READMISSION_90_DAY_IND,FIRST_READMISSION_DATE
PATIENT_ID,,,,,,,,
708b81c9-21a9-411a-aae4-052ef8925b97,cc343294-f037-40c2-a02a-45c36654f268,2010-05-18 21:50:51,23.554214,0,3,0,1,NaN
ad9e95d4-f88b-4544-b5c7-12087fe1fab0,e06a082a-d4e4-48f7-aeef-910bef3deacf,2011-06-26 09:39:47,34.639999,0,8,0,1,NaN
9b5505e9-0806-47ad-8a51-adc39c20b468,2888785f-3852-4e14-ba83-b8e93a56bd01,2018-10-10 08:48:34,19.871729,0,6,0,1,NaN
f9111c77-e28f-47fe-b58e-96641f8d207c,770d1f8a-62f8-4c69-89e3-d308f4f5dc0d,2015-12-27 16:45:31,21.240683,0,4,0,1,NaN
2a9381af-2505-48cb-9299-2bed2d7593f5,9b622d84-7ce2-45c1-abca-68105be048ce,2013-10-19 09:12:24,34.859032,0,2,0,1,NaN


# **Part 3: Export the data to a CSV file**

In [49]:
combine_encounters_patients1.to_csv('EHSAN_SAGHAPOUR.csv')